This notebook will explore use of a simple BERT tokenizer and a simple search to iteratively create groups of four groups of four words from the puzzles. The search will find the group of four words that has the highest average pairwise cosine similarity score and group them. It will then repeat this for the remaining 12 words and so on until 4 groups are created. This will be "zero-shot" in the sense that the BERT model will have no supervision.

# Imports #

In [1]:
import numpy as np
import pandas as pd 
import torch
from transformers import AutoTokenizer, AutoModel
from torch.nn.functional import normalize
from itertools import combinations


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Preprocessing #
Load Connections dataset

In [3]:
df_test = pd.read_csv("/kaggle/input/the-new-york-times-connections/Connections_Data.csv")
df_test.columns
df_test = df_test.drop(['Starting Row', 'Starting Column'], axis=1) #Unecessary info
df_test.head()

,Game ID,Puzzle Date,Word,Group Name,Group Level
0,1,2023-06-12,SNOW,WET WEATHER,0
1,1,2023-06-12,LEVEL,PALINDROMES,3
2,1,2023-06-12,SHIFT,KEYBOARD KEYS,2
3,1,2023-06-12,KAYAK,PALINDROMES,3
4,1,2023-06-12,HEAT,NBA TEAMS,1


## Imputation ##

In [4]:
df_test.isna().sum()

Game ID        0
Puzzle Date    0
Word           2
Group Name     0
Group Level    0
dtype: int64

There seems to be two missing entries from the data. I went to check the actual puzzles from those days to figure out what the missing entries should be and manually entered them. It seems like "NA" from puzzle #62 mistaken got labeled as NaN.

In [5]:
df_test[df_test.Word.isna()]

,Game ID,Puzzle Date,Word,Group Name,Group Level
930,59,2023-08-09,NaN,UNSPECIFIED QUANTITIES,0
978,62,2023-08-12,NaN,PERIODIC TABLE SYMBOLS,3


In [6]:
df_test[(df_test['Game ID'] == 59) & 
    (df_test['Group Name'] == 'UNSPECIFIED QUANTITIES')]

,Game ID,Puzzle Date,Word,Group Name,Group Level
930,59,2023-08-09,NaN,UNSPECIFIED QUANTITIES,0
932,59,2023-08-09,FEW,UNSPECIFIED QUANTITIES,0
937,59,2023-08-09,HANDFUL,UNSPECIFIED QUANTITIES,0
943,59,2023-08-09,SEVERAL,UNSPECIFIED QUANTITIES,0


In [7]:
df_test[(df_test['Game ID'] == 62) & 
    (df_test['Group Name'] == 'PERIODIC TABLE SYMBOLS')]

,Game ID,Puzzle Date,Word,Group Name,Group Level
978,62,2023-08-12,NaN,PERIODIC TABLE SYMBOLS,3
981,62,2023-08-12,NI,PERIODIC TABLE SYMBOLS,3
983,62,2023-08-12,HE,PERIODIC TABLE SYMBOLS,3
987,62,2023-08-12,FE,PERIODIC TABLE SYMBOLS,3


In [8]:
def load_data():
    df = pd.read_csv("/kaggle/input/the-new-york-times-connections/Connections_Data.csv")
    df = df[['Game ID', 'Word', 'Group Name']]
    df.at[930, "Word"] = "SOME"
    df.at[978, "Word"] = "NA"
    return df

In [9]:
df_test = load_data()
df_test.isna().sum()

Game ID       0
Word          0
Group Name    0
dtype: int64

# Encoding #

In this section, the words will be encoded using google's bert tokenizer and model. I also added gpu-acceleration support

In [10]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
model = AutoModel.from_pretrained('google-bert/bert-base-uncased')
model = model.to(device)

#Embeds a list of strings 
def embed_words(words, model):
    inputs = tokenizer(words, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    #Don't compute gradients for embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

2026-02-15 10:48:40.270359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771152520.458276      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771152520.514842      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771152520.965749      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771152520.965784      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771152520.965787      24 computation_placer.cc:177] computation placer alr

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
words = df_test['Word'].tolist()
embeddings = embed_words(words, model)
df_test["Embedding"] = list(embeddings.cpu())


In [12]:
df_test.head()

,Game ID,Word,Group Name,Embedding
0,1,SNOW,WET WEATHER,"[tensor(-0.2204), tensor(0.2819), tensor(-0.13..."
1,1,LEVEL,PALINDROMES,"[tensor(-0.2300), tensor(0.1055), tensor(0.123..."
2,1,SHIFT,KEYBOARD KEYS,"[tensor(-0.1224), tensor(0.0911), tensor(0.119..."
3,1,KAYAK,PALINDROMES,"[tensor(-0.5861), tensor(0.0035), tensor(-0.31..."
4,1,HEAT,NBA TEAMS,"[tensor(-0.4224), tensor(0.1301), tensor(-0.51..."


# Predictions #

I initially tried to use KMeans to select groups but realized there's no easy way to ensure groups of four. Creating four clusters doesn't ensure each has four data points. <br/>
Instead, I created a function that calculates the average cosine similarities between all possible combinations of 4 and chooses the best one. It iteratively repeats this for the remaining 12 and then 8 words.

In [13]:
N_GROUPS = 4 # Number of connections groups
GROUP_SIZE = 4 #Number of words in each connection group

#Takes np.darray or torch tensor as input
#converts to torch tensor to compute mean cosine similarity score between all elements
def get_similarity(embeddings): 
    embeddings = torch.tensor(embeddings)
    X = normalize(embeddings, dim=1)
    sims = X @ X.T 
    #Mask out diagonals (self similarity scores)
    n = sims.size(0)
    mask = ~torch.eye(n, dtype=bool)
    sims = sims[mask]
    return sims.mean()

#Returns array of tuples where each tuple contains 4 indices of a group. Takes numpy embeddings as input
def predict(embeddings):
    preds = []
    sims = []
    remaining_indices = list(range(len(embeddings)))
    
    for i in range(N_GROUPS):
        highest_sim = 0
        best_combo = None
        
        idx_combos = combinations(remaining_indices, GROUP_SIZE)
        for idx_combo in idx_combos:
            candidate_group = embeddings[list(idx_combo)]
            candidate_sim = get_similarity(candidate_group)
            
            if candidate_sim > highest_sim:
                highest_sim = candidate_sim
                best_combo = idx_combo

        preds.append(best_combo)
        sims.append(highest_sim)
        remaining_indices = [i for i in remaining_indices if i not in best_combo]
    

    return preds, sims

#Function gets embeddings from dataframe given game_id
def get_embeddings(df, id):
    return np.stack(df[df['Game ID'] == id]['Embedding'].to_numpy())

In [14]:
TEST_ID = 505
#df_test[df_test['Game ID'] == TEST_ID]

In [15]:
preds, sims = predict(get_embeddings(df_test, TEST_ID))
print(sims)
df_test[df_test['Game ID'] == TEST_ID].iloc[list(preds[0])]

[tensor(0.9862), tensor(0.9585), tensor(0.9387), tensor(0.8766)]


,Game ID,Word,Group Name,Embedding
7985,505,JACKET,YELLOW ___,"[tensor(-0.1677), tensor(0.2440), tensor(0.095..."
7987,505,HAZE,PURPLE ___,"[tensor(-0.0481), tensor(0.1532), tensor(0.144..."
7989,505,JAY,BLUE ___,"[tensor(-0.0280), tensor(0.2684), tensor(0.077..."
7991,505,CAB,YELLOW ___,"[tensor(-0.0451), tensor(0.1967), tensor(0.108..."


# Evaluation #

For all game puzzles, we'll use arrays of tuples to represent groupings. Each tuple will contain the indices of each word. An example representation would be [(1,4,15,14), (5,7,11,12), (2,6,8,10), (0,3,9,13)]

In [16]:
#scores single puzzle given predictions and testing array
def score_puzzle(Y_test, preds):
    n_correct = 0
    for pred in preds:
        if pred in Y_test:
            n_correct += 1
    return n_correct